In [1]:
from vpython import *
import math

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
node_list = [{"id":"a","x":543,"y":323,"z":433,"lock":True},{"id":"b","x":324,"y":530,"z":342,"lock":False},{"id":"c","x":654,"y":392,"z":214,"lock":False},{"id":"d","x":245,"y":250,"z":123,"lock":False}, {"id":"e","x":123,"y":234,"z":345,"lock":False}]
edge_list = [{"source":"a", "target":"b"}, {"source":"a", "target":"c"}, {"source":"a", "target":"d"}, {"source":"b", "target":"d"}, {"source":"c","target":"e"}]

natLength = 200
ks = 0.3
kg = 90
maxRepulsion = 300
drawnNodes = []
drawnLines = []
def draw_nodes(list):
    for drawN in list:
        xcoords = drawN["x"]
        ycoords = drawN["y"]
        zcoords = drawN["z"]
        id = sphere(pos=vector(xcoords,ycoords,zcoords),radius=10,color=color.green, make_trail = True)
        drawnNodes.append(drawN["id"])
        drawnNodes.append(id)
def draw_edge(list):
    for drawE in list:
        for node in node_list:
            if drawE["source"] == node["id"]:
                sourceX = node["x"]
                sourceY = node["y"]
                sourceZ = node["z"]
            if drawE["target"] == node["id"]:
                targetX = node["x"]
                targetY = node["y"]
                targetZ = node["z"]     
        id = arrow(pos = vector(sourceX, sourceY, sourceZ), axis = vector(targetX-sourceX, targetY-sourceY, targetZ-sourceZ), shaftwidth = 5)
        drawnLines.append(id)               

def draw(nodes, edges):
    draw_edge(edges)
    draw_nodes(nodes)

In [3]:
def move(x,y,z,id):
        xPos = node_list[int(id/2)]["x"]
        yPos = node_list[int(id/2)]["y"]
        zPos = node_list[int(id/2)]["z"]
        newxPos = xPos+x
        newyPos = yPos+y
        newzPos = zPos+z
        drawnNodes[id+1].pos=vector(newxPos,newyPos,newzPos)
        node_list[int(id/2)]["x"] = newxPos
        node_list[int(id/2)]["y"] = newyPos
        node_list[int(id/2)]["z"] = newzPos

def move_edges():
    for i in range(len(edge_list)):
        idNum = 0
        for nodes in node_list:
            if nodes["id"] == edge_list[i]["source"]:
                sourceId = idNum
            if nodes["id"] == edge_list[i]["target"]:
                targetId = idNum
            idNum = idNum + 1
        sourcex = node_list[sourceId]["x"]
        sourcey = node_list[sourceId]["y"]
        sourcez = node_list[sourceId]["z"]
        targetx = node_list[targetId]["x"] - node_list[sourceId]["x"]
        targety = node_list[targetId]["y"] - node_list[sourceId]["y"]
        targetz = node_list[targetId]["z"] - node_list[sourceId]["z"]
        drawnLines[i].pos = vector(sourcex,sourcey,sourcez) 
        drawnLines[i].axis = vector(targetx, targety, targetz)

In [4]:
def hookesLaw(a,b):
    xdist = b["x"]-a["x"]
    ydist = b["y"]-a["y"]
    zdist = b["z"]-a["z"]
    d3 = xdist*xdist+ydist*ydist+zdist*zdist
    d3 = math.sqrt(d3)
    distance = d3-natLength
    if d3 == 0:
        return [0,0,0]
    if d3 != 0:
        force = ks*distance/d3
    return [xdist*force, ydist*force,zdist*force ]

def coulombsLaw(a,b):
    xdist = b["x"]-a["x"]
    ydist = b["y"]-a["y"]
    zdist = b["z"]-a["z"]
    d3 = xdist*xdist+ydist*ydist+zdist*zdist
    d3 = math.sqrt(d3)
    if d3 == 0 or d3 > maxRepulsion:
        return [0,0,0]
    if d3 != 0 and d3 < maxRepulsion:
        force = kg/(d3*d3)
    return [-force*xdist,-force*ydist,-force*zdist]


In [5]:
def loop():

    while True:
        sleep(0.05)
        for node1 in node_list:
            netForceX = 0
            netForceY = 0
            netForceZ = 0
            if node1["lock"] == False:
                for node2 in node_list:
                    if node1["id"] != node2["id"]:
                        addedXCForce = coulombsLaw(node1,node2)[0]
                        addedYCForce = coulombsLaw(node1,node2)[1]
                        addedZCForce = coulombsLaw(node1,node2)[2]
                        netForceX = netForceX + addedXCForce
                        netForceY = netForceY + addedYCForce
                        netForceZ = netForceZ + addedZCForce
                for i in range(len(edge_list)):
                    if node1["id"] == edge_list[i]["source"]:
                        for search in node_list:
                            if edge_list[i]["target"] == search["id"]:   
                                addedXHForce = hookesLaw(node1, search)[0]
                                addedYHForce = hookesLaw(node1, search)[1]
                                addedZHForce = hookesLaw(node1, search)[2]
                                netForceX = netForceX + addedXHForce
                                netForceY = netForceY + addedYHForce
                                netForceZ = netForceZ + addedZHForce
                    if node1["id"] == edge_list[i]["target"]:
                        for search in node_list:
                            if edge_list[i]["source"] == search["id"]:  
                                addedXHForce = hookesLaw(node1, search)[0]
                                addedYHForce = hookesLaw(node1, search)[1]
                                addedZHForce = hookesLaw(node1, search)[2]
                                netForceX = netForceX + addedXHForce
                                netForceY = netForceY + addedYHForce
                                netForceZ = netForceZ + addedZHForce
#Draw stuff here
                for idNum in range(len(drawnNodes)):
                    if drawnNodes[idNum] == node1["id"]:
                        move(netForceX,netForceY,netForceZ,idNum)
        move_edges()


In [ ]:
draw(node_list,edge_list)
loop()


#while True:
#    sleep(0.02)
#    for node1 in node_list:
#        netForceX = 5
#       netForceY = 5
#        netForceZ = 5
#        if node1["lock"] == False:
#            for idNum in range(len(drawnNodes)):
#                if drawnNodes[idNum] == node1["id"]:
#                    move(netForceX,netForceY,netForceZ,idNum)